In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
%matplotlib inline
import plotly
from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='ballmdr', api_key='oSgf6vHed7evuaiXDdBb')

import fxcmpy
con = fxcmpy.fxcmpy(config_file = '/Users/ballmdr/blockchain-python/FXCM/fxcm.cfg')

def z(df):
    return (df - df.mean()) / df.std()

In [2]:
symbols = ['EUR/USD']

In [21]:
df = dict()
start = datetime(2011, 1, 1)
end = datetime(2018,12, 31)
for symbol in symbols:
    df[symbol] = con.get_candles(symbol, period='D1', start=start, end=end)
    df[symbol]['Open'] = (df[symbol]['bidopen'] + df[symbol]['askopen']) / 2
    df[symbol]['High'] = (df[symbol]['bidhigh'] + df[symbol]['askhigh']) / 2
    df[symbol]['Low'] = (df[symbol]['bidlow'] + df[symbol]['asklow']) / 2
    df[symbol]['Close'] = (df[symbol]['bidclose'] + df[symbol]['askclose']) / 2
    df[symbol].drop(['bidopen', 'askopen', 'bidhigh', 'askhigh', 'bidlow', 'asklow', 'bidclose', 'askclose'],axis=1, inplace=True)
    df[symbol]['y'] = np.where(df[symbol]['Close'].shift(-1) >= df[symbol]['Close'], 1, 0)

In [22]:
df[symbols[0]].head()

,tickqty,Open,High,Low,Close,y
date,,,,,,
2011-01-03 22:00:00,14070,1.339190,1.339530,1.334950,1.336030,0
2011-01-04 22:00:00,103911,1.336030,1.343345,1.329145,1.330745,0
2011-01-05 22:00:00,115199,1.330745,1.332535,1.312555,1.315050,0
2011-01-06 22:00:00,103240,1.315050,1.317045,1.299755,1.300270,0
2011-01-07 22:00:00,98674,1.300270,1.302115,1.290565,1.291115,1


In [23]:
df[symbols[0]].tail()

,tickqty,Open,High,Low,Close,y
date,,,,,,
2018-12-26 22:00:00,165742,1.140155,1.141430,1.134295,1.135295,1
2018-12-27 22:00:00,430940,1.135295,1.145425,1.135250,1.143070,1
2018-12-28 22:00:00,336745,1.143065,1.147270,1.142695,1.144240,0
2018-12-30 22:00:00,110,1.144705,1.144745,1.143720,1.143825,1
2018-12-31 22:00:00,234863,1.143825,1.146615,1.142135,1.144505,0


In [25]:
symbol_focus = symbols[0]
X = df[symbol_focus].drop('y', axis=1)
y = pd.DataFrame(df[symbol_focus]['y'])
split = int(len(df[symbol_focus]) * 0.60)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [28]:
from sklearn.svm import SVC

In [29]:
model = SVC()

In [30]:
model.fit(X_train, y_train)

/anaconda2/envs/fxcm/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/anaconda2/envs/fxcm/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning:

The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.



SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [34]:
predicts = model.predict(X_test)

In [33]:
from sklearn.metrics import classification_report, confusion_matrix

In [35]:
print(classification_report(y_test, predicts))

              precision    recall  f1-score   support

           0       0.51      0.95      0.66       485
           1       0.46      0.04      0.08       468

   micro avg       0.50      0.50      0.50       953
   macro avg       0.48      0.50      0.37       953
weighted avg       0.48      0.50      0.38       953



In [36]:
print(confusion_matrix(y_test, predicts))

[[460  25]
 [447  21]]


In [37]:
from sklearn.model_selection import GridSearchCV

In [42]:
param_grid = {'C': [0.1,1, 10, 100, 1000], 'gamma': [1,0.1,0.01,0.001,0.0001], 'kernel': ['rbf']} 

In [43]:
grid = GridSearchCV(SVC(), param_grid, verbose=3)
grid.fit(X_train, y_train)

/anaconda2/envs/fxcm/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning:

You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/anaconda2/envs/fxcm/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
/anaconda2/envs/fxcm/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
/anaconda2/envs/fxcm/lib/python3.7/si

Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV]  C=0.1, gamma=1, kernel=rbf, score=0.5031446540880503, total=   0.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV]  C=0.1, gamma=1, kernel=rbf, score=0.5021008403361344, total=   0.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV]  C=0.1, gamma=1, kernel=rbf, score=0.5021008403361344, total=   0.0s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV]  C=0.1, gamma=0.1, kernel=rbf, score=0.5031446540880503, total=   0.0s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV]  C=0.1, gamma=0.1, kernel=rbf, score=0.5021008403361344, total=   0.0s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV]  C=0.1, gamma=0.1, kernel=rbf, score=0.5021008403361344, total=   0.0s
[CV] C=0.1, gamma=0.01, kernel=rbf ............................

/anaconda2/envs/fxcm/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/anaconda2/envs/fxcm/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/anaconda2/envs/fxcm/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/anaconda2/envs/fxcm/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/anaconda2/envs/fxcm/lib/python3.7/site-

[CV]  C=1, gamma=1, kernel=rbf, score=0.5073375262054507, total=   0.0s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV]  C=1, gamma=1, kernel=rbf, score=0.5021008403361344, total=   0.0s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV]  C=1, gamma=1, kernel=rbf, score=0.49789915966386555, total=   0.0s
[CV] C=1, gamma=0.1, kernel=rbf ......................................
[CV]  C=1, gamma=0.1, kernel=rbf, score=0.5136268343815513, total=   0.0s
[CV] C=1, gamma=0.1, kernel=rbf ......................................
[CV]  C=1, gamma=0.1, kernel=rbf, score=0.5147058823529411, total=   0.0s
[CV] C=1, gamma=0.1, kernel=rbf ......................................
[CV]  C=1, gamma=0.1, kernel=rbf, score=0.5126050420168067, total=   0.0s
[CV] C=1, gamma=0.01, kernel=rbf .....................................
[CV]  C=1, gamma=0.01, kernel=rbf, score=0.5366876310272537, total=   0.0s
[CV] C=1, gamma=0.01, kernel=rbf ...........................

/anaconda2/envs/fxcm/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/anaconda2/envs/fxcm/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/anaconda2/envs/fxcm/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/anaconda2/envs/fxcm/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/anaconda2/envs/fxcm/lib/python3.7/site-

...

[CV]  C=1000, gamma=0.0001, kernel=rbf, score=0.5220125786163522, total=   0.2s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV]  C=1000, gamma=0.0001, kernel=rbf, score=0.5084033613445378, total=   0.1s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................


/anaconda2/envs/fxcm/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/anaconda2/envs/fxcm/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[CV]  C=1000, gamma=0.0001, kernel=rbf, score=0.5126050420168067, total=   0.2s


[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    3.0s finished
/anaconda2/envs/fxcm/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'kernel': ['rbf']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [44]:
grid.best_params_

{'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}

In [45]:
grid.best_estimator_

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [46]:
grid_predicts = grid.predict(X_test)

In [47]:
print(classification_report(y_test, grid_predicts))

              precision    recall  f1-score   support

           0       0.52      0.91      0.66       485
           1       0.55      0.11      0.18       468

   micro avg       0.52      0.52      0.52       953
   macro avg       0.53      0.51      0.42       953
weighted avg       0.53      0.52      0.43       953



In [48]:
print(confusion_matrix(y_test, grid_predicts))

[[442  43]
 [416  52]]
